In [1]:
#dependency
import os 
import json
import petpy
import urllib
import time
import pandas as pd

import time
import giphy_client
from giphy_client.rest import ApiException
from pprint import pprint

#database 
from sqlalchemy import create_engine

#
from config import key, secret, api_key 
import config


## Petfinder API

In [ ]:
def get_cats():
    # initialized
    pf = petpy.Petfinder(key=key, secret=secret)
    
    #params
    after_date = '2021-01-01'
    before_date = '2021-04-18'
    animal_type = 'cat'
    status = 'adoptable'
    location = 'Miami, FL'
    distance = '10'
    results_per_page =5
    pages = 2
    
    
    #api call
    api_res = pf.animals(animal_type=animal_type, status=status, location=location, distance=distance,
                     after_date=after_date, before_date=before_date,
                     results_per_page=results_per_page, pages=pages, return_df=True)
    
    # filtered the data few columns 
    new_cat_df = api_res[['id','species','age','gender','animal_id','contact.address.city','published_at']]
    new_cat_df
    
    #reset index to id
    new_cat_df = new_cat_df.set_index('id')

    #rename columns
    new_cat_df = new_cat_df.rename(columns={'contact.address.city':'city', 'published_at':'date_published'})
   
    #--------Database Connection----------
    # Create a SQL Database connection
    connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    # checking tables
    engine.table_names()
    
    try:
        new_cat_df.to_sql(name="stage_cat_db", con=engine, if_exists="append", index=True)
        print("Data loaded successfully")
    
    except:
         print("Data has already been loaded to db")
    
    
    
    return 
get_cats()



In [ ]:
#data collection and storing into the database
def get_dogs():
    
    # initialized
    pf = petpy.Petfinder(key=key, secret=secret)
    
    #params
    after_date = '2021-01-01'
    before_date = '2021-04-18'
    animal_type = 'dog'
    status = 'adoptable'
    location = 'Miami, FL'
    distance = '50'
    results_per_page = 1
    pages = 20
    
    
    #api call
    api_res = pf.animals(animal_type=animal_type, status=status, location=location, distance=distance,
                     after_date=after_date, before_date=before_date,
                     results_per_page=results_per_page, pages=pages, return_df=True)
    
    # filtered the data few columns 
    new_dog_df = api_res[['id','species','age','gender','animal_id','contact.address.city','published_at']]
    #reset index to id
    new_dog_df = new_dog_df.set_index('id')
    
    #rename columns
    new_dog_df = new_dog_df.rename(columns={'contact.address.city':'city', 'published_at':'date_published'})
    
    
  #--------Database Connection----------
    # Create a SQL Database connection
    connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    # checking tables
    engine.table_names()
    
    try:
        new_dog_df.to_sql(name="stage_dog_db", con=engine, if_exists="append", index=True)
        print("Data loaded successfully")
    
    except:
        print("Data has already been loaded to db")

    
    return 
get_dogs()

### Giphy API 

In [ ]:
#dependency
import giphy_client
from giphy_client.rest import ApiException
from pprint import pprint

In [ ]:
def get_giphy():
    #params
    api_instance = giphy_client.DefaultApi()
    api_key = config.api_key 
    limit = 100
    #increase offset by 100 each run 
    offset = 200 
    
    
    #empty list
    gif_url = []
    slug = []
    gif_id = []
    trending_datetime= []

    try: 
        # trending endpoint
        api_response = api_instance.gifs_trending_get(api_key, limit=limit)
        #pprint(api_response.data)

        api_res = api_response.data

        # iterate over the api response 
        for item in api_res:
#             print(item)

            gif_id.append(item.id)
            slug.append(item.slug)
            trending_datetime.append(item.trending_datetime)
            gif_url.append(item.bitly_gif_url)
            

    except ApiException as e:
        print("Exception when calling DefaultApi->gifs_search_get: %s\n" % e)
        
    
    
    #create df for the list items (slug and gif_url)
    giphy_df = pd.DataFrame({'slug':slug,
                            'gif_url':gif_url,'gif_id':gif_id,
                            'trending_datetime':trending_datetime})
    
    #removing duplicate times with value of '0000-00-00 00:00:00'
    giphy_df.drop_duplicates(subset ="trending_datetime",
                     keep = False, inplace = True)
    
    #reset index to gif_url
    giphy_df = giphy_df.set_index('gif_id')

    
    #--------Database Connection----------
    # Create a SQL Database connection
    connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
    engine = create_engine(f'postgresql://{connection_string}')
        
    # checking tables
    engine.table_names()
    
    try:
        giphy_df.to_sql(name="stage_giphy_db", con=engine, if_exists="append", index=True)
        #print("Data loaded successfully")
    
    except:
        print("Data has already been loaded to db")
        
    return #trending_datetime
get_giphy()

In [ ]:
#data validation stet
def update_db():
    
    # Create a SQL Database connection
    connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    #check for connection
#     engine.table_names()
    
    # handle the any duplicates from staging db to final dog_db, cat_db 
    engine.execute(
        """INSERT INTO dog_db 
        SELECT DISTINCT * FROM stage_dog_db
        ON CONFLICT (id) DO UPDATE SET
            species = EXCLUDED.species,
            age = EXCLUDED.age,
            gender = EXCLUDED.gender,
            animal_id = EXCLUDED.animal_id,
            city = EXCLUDED.city,
            date_published = EXCLUDED.date_published""")
    
    engine.execute(
        """INSERT INTO cat_db 
        SELECT DISTINCT * FROM stage_cat_db
        ON CONFLICT (id) DO UPDATE SET
            species = EXCLUDED.species,
            age = EXCLUDED.age,
            gender = EXCLUDED.gender,
            animal_id = EXCLUDED.animal_id,
            city = EXCLUDED.city,
            date_published = EXCLUDED.date_published""")
    
    engine.execute(
        """INSERT INTO giphy_db 
        SELECT DISTINCT(*) 
        FROM stage_giphy_db
        WHERE slug LIKE '%cat%' OR slug LIKE '%dog'
            ON CONFLICT (gif_id) DO UPDATE SET
            slug = EXCLUDED.slug,
            gif_url = EXCLUDED.gif_url,
            trending_datetime = EXCLUDED.trending_datetime
        """)
    
    return
update_db()

## Data Analyst

In [14]:
#plt lib 
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import inspect
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base


In [12]:
def make_Daliy_Chart():
    
    # Create a SQL Database connection
    connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
    engine = create_engine(f'postgresql://{connection_string}')

    #check for connection to db and show all table names 
    engine.table_names()
    
    # reflect an existing database 
    Base = automap_base()

    # reflect the tables
    Base.prepare(engine, reflect=True)

    #view all of the tables with pkey 
    Base.classes.keys()
    
    # Save references to each table in the db
    cat_db = Base.classes.cat_db
    dog_db = Base.classes.dog_db
    giphy_db = Base.classes.giphy_db
    
    # Create(link) btw Python and DB
    session = Session(engine)
    
    #Inspector to explore the database and print the table names
    inspector = inspect(engine)
    inspector.get_table_names()
    
    # Use Inspector to print the column names and types
    cat_table = inspector.get_columns('cat_db')

    dog_table = inspector.get_columns('dog_db')

    giphy_table = inspector.get_columns('giphy_db')

    
    #query the db for 
    #Giphy Daily Trending Dog post
    #Giphy Daily Trending Cat post
    #Pet adoption Cat and Dog Trending 
    
    #---------Cat adoption stats-----------
    cat_adoption = engine.execute("""SELECT COUNT(species) as daily_vol, 
    cast(cast(date_published as date) as varchar) 
    FROM cat_db
    WHERE date_published >= '2021-01-01' AND date_published < '2021/04/18'
    Group by cast(date_published as date)
    """).fetchall()


    cat_adop_dates = [x[1] for x in cat_adoption]
    cat_adop_count = [x[0] for x in cat_adoption]
    
    
#     pprint(f' Dates for CAT Adop {cat_adop_dates}')
#     pprint(f' Count CAT Adop {cat_adop_count}')


 #---------Dog adoption stats-----------

    dog_adoption = engine.execute("""SELECT COUNT(species) as daily_vol, 
    cast(cast(date_published as date) as varchar) 
    FROM dog_db
    WHERE date_published >= '2021-01-01' AND date_published < '2021/04/18'
    Group by cast(date_published as date)
    """).fetchall()
    #list comprehension
    dog_adop_dates = [x[1] for x in dog_adoption]
    dog_adop_count = [x[0] for x in dog_adoption]

#     pprint(f' Dates for DOG Adop {dog_adop_dates}')
#     pprint(f' Count DOG Adop {dog_adop_count}')


 #---------giphy trends stats-----------
    
#     pet_trends = engine.execute("""
#     SELECT COUNT(*) as daily_vol, 
#     cast(cast(trending_datetime as date) as varchar) 
#     FROM giphy_db 
#     Group by cast(trending_datetime as date)
#     """).fetchall()

#     #list comprehension
#     giphy_adop_dates = [x[1] for x in pet_trends]
#     giphy_adop_count = [x[0] for x in pet_trends]

#     print(giphy_adop_dates)
#     print(giphy_adop_count)
    
    ###----------------Make Graphs----------
    
    cat_adoption = pd.DataFrame({'Cat Adoption': cat_adop_count,
                             'Date Trending': cat_adop_dates})
    
#     print(len(cat_adoption))
    
    dog_adoption = pd.DataFrame({'Dog Adoption': dog_adop_count,
                         'Date Trending': dog_adop_dates})
    
#     print(len(dog_adoption))
    
    #new df for cats and dogs to make the graph
    
    combine_adoption = pd.concat([cat_adoption, dog_adoption], axis=1, join='inner')
    print(combine_adoption)
    
#     pet_post = pd.DataFrame({'Pet Post': giphy_adop_count,
#                              'Date Trending': giphy_adop_dates})
    
    
#     fig, ax1 = plt.subplots(figsize=(15, 10))
    
#     #bar graph is for the pet trends posts
#     pet_post['Pet Post'].plot(kind='bar', color='y')
    
#     #line graph is for Cats and Dog adoptions numbers
#     cat_adoption['Cat Adoption'].plot(kind='line', marker='d', secondary_y=True)
#     cat_adoption['Date Trending'].plot(kind='line', marker='d', secondary_y=True)

    
    

make_Daliy_Chart()

    Cat Adoption Date Trending  Dog Adoption Date Trending
0              1    2021-03-09             2    2021-04-04
1              3    2021-04-15             1    2021-02-06
2              1    2021-04-13             1    2021-04-09
3              1    2021-02-14             1    2021-03-14
4              4    2021-04-06             1    2021-04-05
5              4    2021-04-14             1    2021-02-14
6              5    2021-04-12             1    2021-01-16
7              1    2021-04-17             1    2021-03-27
8              2    2021-04-10             1    2021-02-12
9              2    2021-03-25             3    2021-02-09
10             3    2021-02-07             2    2021-01-08
11             1    2021-04-16             1    2021-02-26
12             3    2021-03-07             3    2021-03-12


In [15]:

# Create a SQL Database connection
connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
engine = create_engine(f'postgresql://{connection_string}')

#check for connection and table names
engine.table_names()


['stage_cat_db',
 'stage_dog_db',
 'cat_db',
 'dog_db',
 'stage_giphy_db',
 'giphy_db']

In [16]:
# reflect an existing database 
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

#view all of the classes 
Base.classes.keys()


['cat_db', 'dog_db', 'giphy_db']

In [17]:
# Save references to each table
cat_db = Base.classes.cat_db
dog_db = Base.classes.dog_db
giphy_db = Base.classes.giphy_db

In [18]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [19]:
#Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['stage_cat_db',
 'stage_dog_db',
 'cat_db',
 'dog_db',
 'stage_giphy_db',
 'giphy_db']

In [20]:
# Use Inspector to print the column names and types
cat_table = inspector.get_columns('cat_db')
for c in cat_table:
    print(c['name'], c["type"])

id INTEGER
species VARCHAR(4)
age VARCHAR(10)
gender VARCHAR(6)
animal_id INTEGER
city VARCHAR
date_published TIMESTAMP WITHOUT TIME ZONE


In [21]:
dog_table = inspector.get_columns('dog_db')
for c in dog_table:
    print(c['name'], c["type"])

id INTEGER
species VARCHAR(4)
age VARCHAR(10)
gender VARCHAR(6)
animal_id INTEGER
city VARCHAR
date_published TIMESTAMP WITHOUT TIME ZONE


In [22]:
giphy_table = inspector.get_columns('giphy_db')
for c in giphy_table:
    print(c['name'], c["type"])

gif_id VARCHAR(255)
slug VARCHAR(255)
gif_url VARCHAR(255)
trending_datetime TIMESTAMP WITHOUT TIME ZONE


In [31]:
    # query the db for 
    #Giphy Daily Trending Dog post
    #Giphy Daily Trending Cat post
    #Pet addopption Cat and Dog Trending 
cat_adoption = engine.execute("""SELECT COUNT(species) as daily_vol, 
cast(cast(date_published as date) as varchar) 
FROM cat_db
WHERE date_published >= '2021-01-01' AND date_published < '2021/04/18'
Group by cast(date_published as date)
""").fetchall()

c_adop_dates = [x[1] for x in cat_adoption]
c_adop_count = [x[0] for x in cat_adoption]

# print(cat_adoption)
print(len(c_adop_dates))
print(len(c_adop_count))

0
0


In [29]:
dog_adoption = engine.execute("""SELECT COUNT(species) as daily_vol, 
cast(cast(date_published as date) as varchar) 
FROM dog_db
WHERE date_published >= '2021-01-01' AND date_published < '2021/01/07'
Group by cast(date_published as date)

""").fetchall()
#list comprehension
d_adop_dates = [x[1] for x in dog_adoption]
d_adop_count = [x[0] for x in dog_adoption]

print(len(d_adop_dates))
print(len(d_adop_count))

1
1


In [ ]:
pet_trends = engine.execute("""
SELECT COUNT(*) as daily_vol, 
cast(cast(trending_datetime as date) as varchar) 
FROM giphy_db 
Group by cast(trending_datetime as date)
""").fetchall()

#list comprehension
giphy_adop_dates = [x[1] for x in pet_trends]
giphy_adop_count = [x[0] for x in pet_trends]

print(giphy_adop_dates)
print(giphy_adop_count)

In [ ]:
#need dog post data
dog_trends_date = engine.execute("""SELECT COUNT(species) as daily_vol, 
cast(cast(date_published as date) as varchar) 
FROM cat_db 
Group by cast(date_published as date)
""").fetchall()
dog_trends_date
# for item in dog_trends_date:
#     print(item)

In [ ]:
#need dog post data
cat_trends_date = engine.execute('SELECT COUNT(species) as daily_vol, cast(date_published as date) FROM cat_db Group by cast(date_published as date)').fetchall()
cat_trends_date
# for item in cat_trends_date:
#     print(item)

In [ ]:

df = pd.DataFrame({'a': [100, 200, 150, 175],
                   'b': [430, 30, 20, 10]})
fig, ax1 = plt.subplots(figsize=(15, 10))
df['b'].plot(kind='bar', color='y')
df['a'].plot(kind='line', marker='d', secondary_y=True)